In [4]:
import sys
sys.path.append('structural-probes')
from pathlib import Path

from run_experiment import setup_new_experiment_dir, execute_experiment
import yaml
import torch
import pandas as pd
import eval_probes_on_dataset

In [3]:
data_folder = Path("../../..")
cola = data_folder / 'data'/ 'SST-2' / 'original' / 'raw'

In [4]:
def setup_args_and_folder(): 
    CONFIG_FILE = 'example/config/bert_ptb3.yaml'
    EXPERIMENT_NAME = ''
    SEED = 123

    class Object(object):
        pass

    cli_args = Object()
    cli_args.experiment_config = CONFIG_FILE
    cli_args.results_dir = EXPERIMENT_NAME
    cli_args.train_probe = -1
    cli_args.report_results = 1
    cli_args.seed = SEED

    yaml_args = yaml.load(open(cli_args.experiment_config))
    setup_new_experiment_dir(cli_args, yaml_args, cli_args.results_dir)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    yaml_args['device'] = device
    return yaml_args

yaml_args = setup_args_and_folder()

sentences = ["Yet the act is still charming here"]

Constructing new results directory at results/BERT-disk-parse-distance-2020-4-21-11-8-5-352057/


/home/mli/anaconda3/envs/data-mining/lib/python3.6/site-packages/ipykernel_launcher.py:16: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  app.launch_new_instance()


In [5]:
word_dists, word_depths, predicted_edges = eval_probes_on_dataset.report_on_stdin(yaml_args, sentences)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
[demoing]: 1it [00:00, 2129.09it/s]

Constructing TwoWordPSDProbe
Constructing OneWordPSDProbe
tokens ['[CLS]', 'Yet', 'the', 'act', 'is', 'still', 'charming', 'here', '[SEP]']
torch.Size([1, 9, 1024])
defaultdict(<class 'list'>, {0: [1], 1: [2], 2: [3], 3: [4], 4: [5], 5: [6], 6: [7]})


In [6]:
print(word_depths)

print(predicted_edges)

[2.1053205 4.2791424 2.2283788 2.2952678 1.7218056 1.3695142 1.5609764]
[(2, 5), (1, 2), (3, 5), (4, 5), (5, 6), (0, 6)]


# Notes for generating the Tree data for SST
- This is for reading in the Ground Truth trees that is already given to us in SST
- We'll use Stanford's CoreNLP tools
- Run the ReadSentimentDataset `java -mx4g -cp "*" edu.stanford.nlp.sentiment.ReadSentimentDataset -inputDir data/SST-2/original -outputDir tmp/`
  - The ground truth already does subword partitions, so need to account for that

In [1]:
from nltk.tree import Tree

In [2]:
def read_trees(path):
    with open(path) as f:
        tree_lines = f.readlines()
        
    trees = [Tree.fromstring(treeline) for treeline in tree_lines]
    return trees

In [6]:
# read in SST dataset
sst_trees_base = Path('../../../data/SST-2/tree_format/')

gt_train_path = sst_trees_base / 'train.txt'
gt_dev_path = sst_trees_base / 'dev.txt'

# Read into NTLK Trees

gt_train_trees = read_trees(train_path)
gt_dev_trees = read_trees(dev_path)

# Calculate distance between the two trees
# one tree's format is from structural probes
# other tree's format is from the dataset (gt_*_trees)